In [1]:
from Bio import SeqIO
import pandas as pd
from helpers import add_blast_header_to_file

# Extract the protein fasta sequences using the locus identifiers

In [2]:
# file available at https://zenodo.org/record/3712239
with open("ITAG4.0_cDNA.fasta","r") as filin:
    recs = [rec for rec in SeqIO.parse(filin,"fasta")]

In [3]:
precursor_genes_table = pd.read_csv("../targets.tsv",sep="\t")
precursor_genes_table.head(n=3)

,Sequence_ID,Functional.annotation,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,CV,F1,F1_hab,F2_127,F2_151,F2_28,F2_411,F2_445,F2_73,PI127826
0,Solyc10g005840,Farnesyl pyrophosphate synthase,365.199108,2.290849,0.516305,4.437009,0.000009,0.002852,792,728,62,55,521,354,457,601,86,79
1,Solyc10g005820,Farnesyl pyrophosphate synthase,611.355900,-1.788514,0.467578,-3.825060,0.000131,0.016354,366,299,1528,330,217,565,401,134,680,1822
2,Solyc06g066310,Phosphomevalonate kinase,1724.742259,-2.545413,0.678477,-3.751659,0.000176,0.020278,237,722,5804,748,261,632,716,612,1367,6899


In [4]:
precursor_loci = precursor_genes_table["Sequence_ID"].tolist()
precursor_loci[0:2]

['Solyc10g005840', 'Solyc10g005820']

In [5]:
with open("precursor_mrnas.fa","w") as fileout:
    for precursor in precursor_loci:
        for rec in recs:
            if precursor in rec.id:
                fileout.write(">" + rec.id + "\n" + str(rec.seq) + "\n")
    

# BLAT the protein sequences against the habrochaites genome

In [6]:
{
    "tags": [
        "hide_output",
    ]
}
! blat -t=dna -q=dna -minIdentity=90 -out=blast8 precursor_mrnas.fa PI127826.final.fasta output.blast8.tsv

Loaded 69600 letters in 41 sequences
Query sequence Super-Scaffold_67 has size 12772664, it might take a while.
Query sequence Super-Scaffold_71 has size 15961298, it might take a while.
Query sequence Super-Scaffold_78 has size 25664899, it might take a while.
Query sequence Super-Scaffold_79 has size 7533941, it might take a while.
Query sequence Super-Scaffold_84 has size 26914694, it might take a while.
Query sequence Super-Scaffold_100 has size 38267190, it might take a while.
Query sequence Super-Scaffold_107 has size 13361701, it might take a while.
Query sequence Super-Scaffold_109 has size 6463468, it might take a while.
Query sequence Super-Scaffold_128 has size 6569248, it might take a while.
Query sequence Super-Scaffold_171 has size 14400801, it might take a while.
Query sequence Super-Scaffold_210 has size 5393027, it might take a while.
Query sequence Super-Scaffold_299 has size 8137420, it might take a while.
Query sequence Super-Scaffold_313 has size 6065232, it might 

# Convert BLAT results to GFF format
**GFF, or the General Feature Format** is used to describe genes and other features of DNA, RNA and protein sequences. It comes with the .gff extension.  
Fields:
* Column 1 is refseq name: Name of chromosome or scaffold. Chromosomes can be given without the 'chr' prefix.	
* Column 2 is source: Source of annotation, name of program that generated this feature.	
* Column 3 is feature: Feature type name. Gene, variation, similarity	
* Column 4 is start: Start position, starting at 1.	
* Column 5 is end: End position, starting at 1.	
* Column 6 is score: Floating point value. For scores such as similarity, identity, etc.	
* Column 7 is strand: '+' for forward and '-' for reverse.	
* Column 8 is frame: Either 0, 1 or 2.	0 indicates first base of the feature is first base of codon, 1 indicates second base of feature is the first base of a codon, etc.	
* Column 9 is attribute: Semicolon-separated list of tag-value pairs. Provides additional information about each feature
    
To perform this step, I am using the `blast2gff` utility of the MGKit https://mgkit.readthedocs.io/en/0.3.4/scripts/blast2gff.html#options

## Blast2gff options explained

`--no-split`        the sequence header will be used as gene_id  
`--db`              blastdb used (here PI12826.final.fasta)
`--feat-type`       feature type to be used in the GFF file (exon)
`output.blast8.tsv` Blast file name (input)  
`PI127826.gff`      GFF file name (output)  

In [7]:
%%bash

blast2gff blastdb --no-split \
                  --db-used PI127826.final.fasta \
                  --feat-type exon \
                  output.blast8.tsv \
                  PI127826.gff              

INFO - mgkit.workflow.blast2gff: Writing to file (PI127826.gff)
INFO - mgkit.io.blast: Reading BLAST results from file (output.blast8.tsv)
INFO - mgkit.io.blast: Read 795 BLAST records


In [8]:
pd.read_csv("PI127826.gff",sep="\t",header=None).head(n=3)

,0,1,2,3,4,5,6,7,8
0,Super-Scaffold_67,BLAST,exon,6484348,6484601,477.0,+,0,"bitscore=""477.0"";db=""PI127826.final.fasta"";dbq..."
1,Super-Scaffold_67,BLAST,exon,6488371,6488497,245.0,+,0,"bitscore=""245.0"";db=""PI127826.final.fasta"";dbq..."
2,Super-Scaffold_67,BLAST,exon,6486478,6486600,229.0,+,0,"bitscore=""229.0"";db=""PI127826.final.fasta"";dbq..."


# Small changes for downstream compatibility 
The produced annotation needs to be compatible with the RNA-seq pipeline (that only accepts GTF).

In [24]:
! sed 's/gene_id/transcript_id/g' PI127826.gff > ../PI127826_annotation.gtf

In [25]:
! head -n 3 ../PI127826_annotation.gtf

Super-Scaffold_67	BLAST	exon	6484348	6484601	477.0	+	0	bitscore="477.0";db="PI127826.final.fasta";dbq="10";evalue="7.6e-135";frame="f0";transcript_id="Solyc12g015860.2.1";identity="98.03";subject_end="1065";subject_start="1318";uid="448f69a2-fac0-4b14-8e7d-06282a8bec8e"
Super-Scaffold_67	BLAST	exon	6488371	6488497	245.0	+	0	bitscore="245.0";db="PI127826.final.fasta";dbq="10";evalue="6.7e-65";frame="f0";transcript_id="Solyc12g015860.2.1";identity="100.0";subject_end="144";subject_start="270";uid="7367b446-7653-4a4f-b1ff-dfc313c203f6"
Super-Scaffold_67	BLAST	exon	6486478	6486600	229.0	+	0	bitscore="229.0";db="PI127826.final.fasta";dbq="10";evalue="2.8e-60";frame="f0";transcript_id="Solyc12g015860.2.1";identity="98.37";subject_end="501";subject_start="623";uid="833d0c22-27da-4981-a561-2758af03412e"
